In [1]:
from magma import *
from functools import reduce
import os
os.environ["MANTLE"] = "coreir"
from mantle import eq

def one_hot_mux(conds, inputs):
    outputs = []
    for cond, inp in zip(conds, inputs):
        outputs.append(inp & bits([cond for _ in range(len(inp))]))
    return reduce(lambda x, y: x | y, outputs)


class SimpleALU(Circuit):
    name = "SimpleALU"
    IO = ["a", In(UInt(4)), "b", In(UInt(4)), "opcode", In(UInt(2)), "out", Out(UInt(4))]
    
    @classmethod
    def definition(io):
        is_op0 = eq(io.opcode, uint(0, n=2))
        is_op1 = eq(io.opcode, uint(1, n=2))
        is_op2 = eq(io.opcode, uint(2, n=2))
        is_op3 = eq(io.opcode, uint(3, n=2))
        op0_out = io.a + io.b
        op1_out = io.a - io.b
        op2_out = io.a
        op3_out = io.b
        wire(io.out, one_hot_mux([is_op0, is_op1, is_op2, is_op3], [op0_out, op1_out, op2_out, op3_out]))

In [2]:
# from magma.backend.verilog import compile as compile_verilog

# print(compile_verilog(SimpleALU))

from magma.backend.coreir_ import compile as compile_coreir

compile_coreir(SimpleALU, "build/SimpleALU.json")
with open("build/SimpleALU.json", "r") as f:
    print(f.read())

{"top":"global.SimpleALU",
"namespaces":{
  "global":{
    "modules":{
      "Add4":{
        "type":["Record",{
          "I0":["Array",4,"BitIn"],
          "I1":["Array",4,"BitIn"],
          "O":["Array",4,"Bit"]
        }],
        "instances":{
          "inst0":{
            "genref":"coreir.add",
            "genargs":{"width":["Int",4]}
          }
        },
        "connections":[
          ["inst0.in0","self.I0"],
          ["inst0.in1","self.I1"],
          ["inst0.out","self.O"]
        ]
      },
      "Add4_cin":{
        "type":["Record",{
          "I0":["Array",4,"BitIn"],
          "I1":["Array",4,"BitIn"],
          "O":["Array",4,"Bit"],
          "CIN":"BitIn"
        }],
        "instances":{
          "bit_const_GND":{
            "modref":"corebit.const",
            "modargs":{"value":["Bool",false]}
          },
          "inst0":{
            "genref":"coreir.add",
            "genargs":{"width":["Int",4]}
          },
          "inst1":{
            "genre

In [3]:
from magma.simulator import PythonSimulator
from magma.bit_vector import BitVector

simulator = PythonSimulator(SimpleALU)
simulator.set_value(SimpleALU.a, BitVector(3, num_bits=4))
simulator.set_value(SimpleALU.b, BitVector(2, num_bits=4))
simulator.set_value(SimpleALU.opcode, BitVector(0, num_bits=2))
simulator.evaluate()
assert simulator.get_value(SimpleALU.out) == BitVector(3 + 2, num_bits=4), simulator.get_value(SimpleALU.out)

simulator.set_value(SimpleALU.a, BitVector(3, num_bits=4))
simulator.set_value(SimpleALU.b, BitVector(2, num_bits=4))
simulator.set_value(SimpleALU.opcode, BitVector(1, num_bits=2))
simulator.evaluate()
assert simulator.get_value(SimpleALU.out) == BitVector(3 - 2, num_bits=4)
simulator.set_value(SimpleALU.a, BitVector(3, num_bits=4))
simulator.set_value(SimpleALU.b, BitVector(2, num_bits=4))
simulator.set_value(SimpleALU.opcode, BitVector(2, num_bits=2))
simulator.evaluate()
assert simulator.get_value(SimpleALU.out) == BitVector(3, num_bits=4)

simulator.set_value(SimpleALU.a, BitVector(3, num_bits=4))
simulator.set_value(SimpleALU.b, BitVector(2, num_bits=4))
simulator.set_value(SimpleALU.opcode, BitVector(3, num_bits=2))
simulator.evaluate()
assert simulator.get_value(SimpleALU.out) == BitVector(2, num_bits=4)
print("Success!")

Success!
